In [23]:
import os
from bs4 import BeautifulSoup
import pandas as pd
import requests
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer

In [25]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\valer\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [2]:
url = 'https://www.immobiliare.it/vendita-case/roma/?criterio=rilevanza&pag=1'


In [3]:
response = requests.get(url)

In [4]:
html_soup = BeautifulSoup(response.text, 'html.parser')

In [5]:
house_containers = html_soup.find_all('div', class_ = 'listing-item_body--content')

In [6]:
second_house =house_containers[1]
second_house

<div class="listing-item_body--content">
<p class="titolo text-primary">
<a data-row-link="" href="https://www.immobiliare.it/53131931-Vendita-Bilocale-viale-Italo-Calvino-Roma.html" id="link_ad_53131931" title="Bilocale viale Italo Calvino, Fonte Ostiense, Roma">Bilocale viale Italo Calvino, Fonte Ostiense, Roma</a>
</p>
<ul class="listing-features list-piped">
<li class="lif__item lif__pricing">
                    € 225.000
                </li><li class="lif__item"><div class="lif__data"><span class="text-bold">2 </span><svg class="icon-svg" height="16" width="16"><use xlink:href="#rooms"></use></svg></div><div class="lif__text lif--muted">locali</div></li><li class="lif__item"><div class="lif__data"><span class="text-bold">50</span> m<sup>2</sup></div><div class="lif__text lif--muted">superficie</div></li><li class="lif__item"><div class="lif__data"><span class="text-bold">1 </span><svg class="icon-svg" height="16" width="16"><use xlink:href="#bathrooms"></use></svg></div><div cla

In [7]:
second_title = second_house.a

In [8]:
second_title.text

'Bilocale viale Italo Calvino, Fonte Ostiense, Roma'

In [9]:
second_price = second_house.find_all('li')
second_price[0].text

'\n                    € 225.000\n                '

In [10]:
second_price = second_house.find('li', class_="lif__item lif__pricing").text.strip()
second_price

'€ 225.000'

In [93]:
second_price = second_price.text

In [11]:
second_locals_surface_bathrooms = second_house.findAll('span', class_="text-bold")

In [21]:
second_locals = second_locals_surface_bathrooms[0]

In [22]:
second_locals = second_locals.text.strip()

In [23]:
second_locals

'2'

In [24]:
second_surface = second_locals_surface_bathrooms[1]

In [25]:
second_surface = second_surface.text.strip()

In [26]:
second_surface

'50'

In [27]:
second_bathrooms = second_locals_surface_bathrooms[2]

In [28]:
second_bathrooms = second_bathrooms.text.strip()

In [29]:
second_bathrooms

'1'

In [30]:
second_floor = second_house.findAll('abbr', class_="text-bold im-abbr")

In [31]:
second_floor.strip()

AttributeError: ResultSet object has no attribute 'strip'. You're probably treating a list of items like a single item. Did you call find_all() when you meant to call find()?

In [6]:
# Lists to store the scraped data int
titles = []
prices = []
locals_ = []
surfaces = []
bathrooms = []
floors = []
descriptions = []

In [7]:
for house in house_containers:
    house.find('div', class_ = '"lif__item lif__pricing')
    locals_surface_bathrooms = house.findAll('span', class_="text-bold")
    floor = house.findAll('abbr', class_="text-bold im-abbr")
    # If the house has all info that need us (price, locals, surface, bathroom and floor) then extract:
    if len(locals_surface_bathrooms) == 3 :
                      if  len(floor) == 1:

        # The name
                          
                          title = house.a.text
                          titles.append(title)

        # The price
                          price = house.find('li', class_="lif__item lif__pricing").text.strip()
                          prices.append(price)

        #the number of locals, surface (m^2), and number of bathrooms
                          local_ = locals_surface_bathrooms[0].text.strip()
                          locals_.append(local_)
                          surface = locals_surface_bathrooms[1].text.strip()
                          surfaces.append(surface)
                          bathroom = locals_surface_bathrooms[2].text.strip()
                          bathrooms.append(bathroom)
        
        #the number of floor
                          floors.append(floor)

        # the description
                          description = house.find('p', class_="descrizione__truncate").text.strip()
                          descriptions.append(description)

In [12]:
len(surfaces)

23

In [13]:
test_df = pd.DataFrame({'title': titles,
                       'price': prices,
                       'locals': locals_,
                       'surface': surfaces,
                       'bathroom': bathroom,
                       'floor': floors})

In [202]:
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 6 columns):
title       23 non-null object
price       23 non-null object
locals      23 non-null object
surface     23 non-null object
bathroom    23 non-null object
floor       23 non-null object
dtypes: object(6)
memory usage: 1.2+ KB
None


In [14]:
test_df

,title,price,locals,surface,bathroom,floor
0,"Bilocale viale Italo Calvino, Fonte Ostiense, ...",€ 225.000,2,50,2,"[<abbr class=""text-bold im-abbr"" title=""1"">1 ..."
1,"Appartamento via di Pietralata, Pietralata, Roma",€ 395.000,5+,130,2,"[<abbr class=""text-bold im-abbr"" title=""1"">1 ..."
2,"Appartamento via della Tribuna di Campitelli, ...",€ 3.700.000,5+,250,2,"[<abbr class=""text-bold im-abbr"" title=""5"">5 ..."
3,"Attico / Mansarda via Degli Orti Di Galba, Mon...",€ 850.000,5,175,2,"[<abbr class=""text-bold im-abbr"" title=""Ultimo..."
4,"Trilocale via Fontanellato 75, Ardeatino - Mon...",€ 279.000,3,110,2,"[<abbr class=""text-bold im-abbr"" title=""Piano ..."
5,"Trilocale via Genserico Fontana, 11, Balduina,...",€ 520.000,3,105,2,"[<abbr class=""text-bold im-abbr"" title=""1"">1 ..."
6,"Trilocale via Arnaldo Cantani, Massimina, Roma",€ 269.000,3,87,2,"[<abbr class=""text-bold im-abbr"" title=""3"">3 ..."
7,"Attico / Mansarda via dei Baullari, Campo de' ...",€ 1.050.000,5+,235,2,"[<abbr class=""text-bold im-abbr"" title=""3"">3 ..."
8,"Appartamento via della Farnesina 323, Camilluc...",€ 1.800.000,5+,250,2,"[<abbr class=""text-bold im-abbr"" title=""3"">3 ..."
9,"Attico / Mansarda via Nicolò Piccolomini, Greg...",€ 1.650.000,5+,220,2,"[<abbr class=""text-bold im-abbr"" title=""4"">4 ..."


Now, our goal is create a tf-idf matrix

First step: create a list of all words in houses description

In [39]:
descriptions[0]

'PAPILLO EUR in elegante complesso residenziale rifinitissimo bilocale composto da soggiorno con angolo cottura, stanza da letto bagno e ampio balcone . con Rifiniture di pregio, pavimenti in parquet / grees, infissi in legno con vetro camera e porte in noce, grate nel salone, riscaldamento termoautonomo con caldaia centralizzata, aria condizionata,...'

In [40]:
a = descriptions[0]

In [57]:
def clening_description(description):  # use all the techniques to remove unwanted items from words
    
    tokenizer = RegexpTokenizer(r'\w+')
    ps = PorterStemmer()
    # Remove "\\n" and replace with a space
    description = description.replace("\\n", " ")
    # Convert everything to lowercase
    description = description.lower()
    # Removing punctuation
    description = tokenizer.tokenize(description)
    # Stemming
    description = [ps.stem(word) for word in description]
    # Removing stopwords
    description = [word for word in description if word not in stopwords.words('italian')]
    # No integers
    description = [x for x in description if not (x.isdigit() or x[0].isdigit())]
    return description

In [58]:
a = clening_description(descriptions[0])

In [59]:
a

['papillo',
 'eur',
 'elegant',
 'complesso',
 'residenzial',
 'rifinitissimo',
 'bilocal',
 'composto',
 'soggiorno',
 'angolo',
 'cottura',
 'stanza',
 'letto',
 'bagno',
 'ampio',
 'balcon',
 'rifinitur',
 'pregio',
 'pavimenti',
 'parquet',
 'gree',
 'infissi',
 'legno',
 'vetro',
 'camera',
 'port',
 'noce',
 'grate',
 'salon',
 'riscaldamento',
 'termoautonomo',
 'caldaia',
 'centralizzata',
 'aria',
 'condizionata']